# DGD+IOS算法+小值攻击

In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity_robust1 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust1=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust1 = 0.6
dual_variable_robust1 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust1= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust1=[]#绘图横坐标
A_robust1 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust1_help=[]
q_robust1=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust1=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust1=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust1=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust1=np.zeros(shape=(100,60000))

In [3]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    dual_variable_robust1[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust1*100)),100):
    dual_variable_robust1[i][0]=-600
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust1*100)):
    agent_resource_quantity_robust1[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust1=0
for i in range (0,100-int(proportion_robust1*100)):
    sum_robust1 = sum_robust1 + agent_resource_quantity_robust1[i][0]
average_resource_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

#诚实用户平均成本初始化
sum_robust1=0
for i in range(0,100-int(proportion_robust1*100)):
    sum_robust1=sum_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][0]-b_robust1[i])*(agent_resource_quantity_robust1[i][0]-b_robust1[i])
average_cost_honest_robust1[0][0]=sum_robust1/(100-int(proportion_robust1*100))

iteration_robust1.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust1[i][j]!=0:
            neighbor_robust1_help.append(j)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust1_help=0
    for j in range(0,15):
        if neighbor_robust1[i][j]==94 or neighbor_robust1[i][j]==95 or neighbor_robust1[i][j]==96 or neighbor_robust1[i][j]==97 or neighbor_robust1[i][j]==98 or neighbor_robust1[i][j]==99:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help)       

In [4]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,q_robust1):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+小值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust1[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust1[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust1[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust1[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust1)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust1.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust1=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust1*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1]-iteration_update_robust1*((50/100)-(1/100)*agent_resource_quantity_robust1[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust1*100)),100):
            dual_variable_intermediate_robust1.append(dual_variable_robust1[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust1*100)):
            u_neighbor_help_robust1=neighbor_robust1[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust1.append(i)
            for q in range(0, int(q_robust1[i])):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust1=0
                h_robust1=0
                for j in range(0,len(u_neighbor_help_robust1)):#计算当前循环中的h值
                    h_robust1=h_robust1+W_robust1[i][u_neighbor_help_robust1[j]]
                for j in range(0,len(u_neighbor_help_robust1)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust1=dual_variable_weight_average_robust1+(1/h_robust1)*(W_robust1[i][u_neighbor_help_robust1[j]]*dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust1.remove(i)
                neighbor_index_robust1=u_neighbor_help_robust1[0]
                max_distance_roubust1=abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[0]]-dual_variable_weight_average_robust1)
                for j in range(1,len(u_neighbor_help_robust1)):
                    if abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]]-dual_variable_weight_average_robust1)>=max_distance_roubust1:
                        max_distance_roubust1=abs(dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]]-dual_variable_weight_average_robust1)
                        neighbor_index_robust1=u_neighbor_help_robust1[j]
                u_neighbor_help_robust1.remove(neighbor_index_robust1)
                u_neighbor_help_robust1.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust1=0
            h_robust1=0
            for j in range(0,len(u_neighbor_help_robust1)):
                h_robust1=h_robust1+W_robust1[i][u_neighbor_help_robust1[j]]
            for j in range(0,len(u_neighbor_help_robust1)):
                dual_variable_weight_average_robust1=dual_variable_weight_average_robust1+(1/h_robust1)*(W_robust1[i][u_neighbor_help_robust1[j]]*dual_variable_intermediate_robust1[u_neighbor_help_robust1[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust1[i][k]=dual_variable_weight_average_robust1
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust1)):
                if u_neighbor_help_robust1[j]==94 or u_neighbor_help_robust1[j]==95 or u_neighbor_help_robust1[j]==96 or u_neighbor_help_robust1[j]==97 or u_neighbor_help_robust1[j]==98 or u_neighbor_help_robust1[j]==99:
                    check_robust1[i][k]=1 
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            dual_variable_robust1[i][k]=dual_variable_robust1[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust1*100)):
            agent_resource_quantity_robust1[i][k]=(dual_variable_robust1[i][k]-2*a_robust1[i]*b_robust1[i])/(-2*a_robust1[i])
            #原变量的投影操作
            if agent_resource_quantity_robust1[i][k] < 0:
                agent_resource_quantity_robust1[i][k]=0
            elif agent_resource_quantity_robust1[i][k] > 100: 
                agent_resource_quantity_robust1[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust1*100),100):
            agent_resource_quantity_robust1[i][k]=agent_resource_quantity_robust1[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust1=0
        for i in range (0,100-int(proportion_robust1*100)):
            sum1_robust1 = sum1_robust1 + agent_resource_quantity_robust1[i][k]
        average_resource_honest_robust1[0][k]=sum1_robust1/(100-int(proportion_robust1*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust1=0
        for i in range(0,100-int(proportion_robust1*100)):
            sum2_robust1=sum2_robust1+a_robust1[i]*(agent_resource_quantity_robust1[i][k]-b_robust1[i])*(agent_resource_quantity_robust1[i][k]-b_robust1[i])
        average_cost_honest_robust1[0][k]=sum2_robust1/(100-int(proportion_robust1*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust1[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust1[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust1[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust1[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust1)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust1)
            print("****************************************************************************")'''
        
        iteration_update_robust1=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
Robust_DGD_IOS_robust1(a_robust1,b_robust1,agent_resource_quantity_robust1,average_resource_honest_robust1,average_cost_honest_robust1,iteration_update_robust1,dual_variable_robust1,A_robust1,iteration_robust1,q_robust1)

****************************DGD算法+IOS+小值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.12070683276905
agent 2 在 3999 次迭代中的资源申请量为: 55.95304928256159
agent 3 在 3999 次迭代中的资源申请量为: 60.561504445775654
agent 4 在 3999 次迭代中的资源申请量为: 37.87880941536425
agent 5 在 3999 次迭代中的资源申请量为: 57.485123743591
agent 6 在 3999 次迭代中的资源申请量为: 55.264203791618655
agent 7 在 3999 次迭代中的资源申请量为: 65.53125226251791
agent 8 在 3999 次迭代中的资源申请量为: 46.27086551757551
agent 9 在 3999 次迭代中的资源申请量为: 65.55042218484795
agent 10 在 3999 次迭代中的资源申请量为: 54.14963600043463
agent 11 在 3999 次迭代中的资源申请量为: 65.3573429729549
agent 12 在 3999 次迭代中的资源申请量为: 38.35910883660039
agent 13 在 3999 次迭代中的资源申请量为: 49.63453967492609
agent 14 在 3999 次迭代中的资源申请量为: 41.30791885082795
agent 15 在 3999 次迭代中的资源申请量为: 42.43389362994332
agent 16 在 3999 次迭代中的资源申请量为: 49.18225461624181
agent 17 在 3999 次迭代中的资源申请量为: 50.050423332647604
agent 18 在 3999 次迭代中的资源申请量为: 46.47809367381057
agent 19 在 3999 次迭代中的资源申请量为: 44.719362884828016
agent 20 在 3999 次迭代中的资源申请量为: 50.73037511281795
agent 21 在 3999 次迭代中的资源申请量为: 69.8324590448429
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.17297628108825
agent 2 在 5999 次迭代中的资源申请量为: 56.005249429242454
agent 3 在 5999 次迭代中的资源申请量为: 60.61881408119263
agent 4 在 5999 次迭代中的资源申请量为: 37.91426629347785
agent 5 在 5999 次迭代中的资源申请量为: 57.53863333527171
agent 6 在 5999 次迭代中的资源申请量为: 55.315504287381664
agent 7 在 5999 次迭代中的资源申请量为: 65.59365730930715
agent 8 在 5999 次迭代中的资源申请量为: 46.31365787605212
agent 9 在 5999 次迭代中的资源申请量为: 65.61211513602582
agent 10 在 5999 次迭代中的资源申请量为: 54.200781961994586
agent 11 在 5999 次迭代中的资源申请量为: 65.4202496312116
agent 12 在 5999 次迭代中的资源申请量为: 38.39447317853655
agent 13 在 5999 次迭代中的资源申请量为: 49.68164834947196
agent 14 在 5999 次迭代中的资源申请量为: 41.34596408257298
agent 15 在 5999 次迭代中的资源申请量为: 42.472927353235804
agent 16 在 5999 次迭代中的资源申请量为: 49.22773063371897
agent 17 在 5999 次迭代中的资源申请量为: 50.09649678214453
agent 18 在 5999 次迭代中的资源申请量为: 46.52113818982445
agent 19 在 5999 次迭代中的资源申请量为: 44.76131545941006
agent 20 在 5999 次迭代中的资源申请量为: 50.77720713463902
agent 21 在 5999 次迭代中的资源申请量为: 69.89814844393362
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.175489324357414
agent 2 在 7999 次迭代中的资源申请量为: 56.007965353906926
agent 3 在 7999 次迭代中的资源申请量为: 60.62159158212441
agent 4 在 7999 次迭代中的资源申请量为: 37.91601268471958
agent 5 在 7999 次迭代中的资源申请量为: 57.541255995392
agent 6 在 7999 次迭代中的资源申请量为: 55.31800509270703
agent 7 在 7999 次迭代中的资源申请量为: 65.5966351524103
agent 8 在 7999 次迭代中的资源申请量为: 46.31561158193708
agent 9 在 7999 次迭代中的资源申请量为: 65.61508346873212
agent 10 在 7999 次迭代中的资源申请量为: 54.20359292127057
agent 11 在 7999 次迭代中的资源申请量为: 65.4232930589376
agent 12 在 7999 次迭代中的资源申请量为: 38.39601946339723
agent 13 在 7999 次迭代中的资源申请量为: 49.68379656357857
agent 14 在 7999 次迭代中的资源申请量为: 41.347826259941925
agent 15 在 7999 次迭代中的资源申请量为: 42.47470517393577
agent 16 在 7999 次迭代中的资源申请量为: 49.229785314883415
agent 17 在 7999 次迭代中的资源申请量为: 50.09869212325308
agent 18 在 7999 次迭代中的资源申请量为: 46.523070267456085
agent 19 在 7999 次迭代中的资源申请量为: 44.76311399652334
agent 20 在 7999 次迭代中的资源申请量为: 50.77954075956235
agent 21 在 7999 次迭代中的资源申请量为: 69.90139441831356
agent 2

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.17563138506366
agent 2 在 9999 次迭代中的资源申请量为: 56.00827401335625
agent 3 在 9999 次迭代中的资源申请量为: 60.62176524552095
agent 4 在 9999 次迭代中的资源申请量为: 37.91614275552205
agent 5 在 9999 次迭代中的资源申请量为: 57.541441861572025
agent 6 在 9999 次迭代中的资源申请量为: 55.318172297737725
agent 7 在 9999 次迭代中的资源申请量为: 65.59678655677419
agent 8 在 9999 次迭代中的资源申请量为: 46.31564401731814
agent 9 在 9999 次迭代中的资源申请量为: 65.61525293818953
agent 10 在 9999 次迭代中的资源申请量为: 54.204016573454865
agent 11 在 9999 次迭代中的资源申请量为: 65.42347936737477
agent 12 在 9999 次迭代中的资源申请量为: 38.39599103420858
agent 13 在 9999 次迭代中的资源申请量为: 49.683830204976175
agent 14 在 9999 次迭代中的资源申请量为: 41.347956362274005
agent 15 在 9999 次迭代中的资源申请量为: 42.47473129706811
agent 16 在 9999 次迭代中的资源申请量为: 49.229802355617565
agent 17 在 9999 次迭代中的资源申请量为: 50.09880132319444
agent 18 在 9999 次迭代中的资源申请量为: 46.52307609052926
agent 19 在 9999 次迭代中的资源申请量为: 44.76305130458887
agent 20 在 9999 次迭代中的资源申请量为: 50.77973436797195
agent 21 在 9999 次迭代中的资源申请量为: 69.90164390319808
ag

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.17570583156672
agent 2 在 51999 次迭代中的资源申请量为: 56.00947727472043
agent 3 在 51999 次迭代中的资源申请量为: 60.621968159243025
agent 4 在 51999 次迭代中的资源申请量为: 37.916421343091805
agent 5 在 51999 次迭代中的资源申请量为: 57.54179180475173
agent 6 在 51999 次迭代中的资源申请量为: 55.318433286222785
agent 7 在 51999 次迭代中的资源申请量为: 65.59675219664979
agent 8 在 51999 次迭代中的资源申请量为: 46.31513726231826
agent 9 在 51999 次迭代中的资源申请量为: 65.61535303908309
agent 10 在 51999 次迭代中的资源申请量为: 54.20601608468502
agent 11 在 51999 次迭代中的资源申请量为: 65.42367283460275
agent 12 在 51999 次迭代中的资源申请量为: 38.39519839555732
agent 13 在 51999 次迭代中的资源申请量为: 49.683258376913756
agent 14 在 51999 次迭代中的资源申请量为: 41.34819124208708
agent 15 在 51999 次迭代中的资源申请量为: 42.474245630962805
agent 16 在 51999 次迭代中的资源申请量为: 49.229145830464255
agent 17 在 51999 次迭代中的资源申请量为: 50.09875848585519
agent 18 在 51999 次迭代中的资源申请量为: 46.522384945319395
agent 19 在 51999 次迭代中的资源申请量为: 44.76191493665629
agent 20 在 51999 次迭代中的资源申请量为: 50.78024996086717
agent 21 在 51999 次迭代中的资源申请量

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.17570725644687
agent 2 在 53999 次迭代中的资源申请量为: 56.00950242610763
agent 3 在 53999 次迭代中的资源申请量为: 60.62197227313143
agent 4 在 53999 次迭代中的资源申请量为: 37.91642710284778
agent 5 在 53999 次迭代中的资源申请量为: 57.54179901794708
agent 6 在 53999 次迭代中的资源申请量为: 55.31843863262337
agent 7 在 53999 次迭代中的资源申请量为: 65.59675130679224
agent 8 在 53999 次迭代中的资源申请量为: 46.31512649564598
agent 9 在 53999 次迭代中的资源申请量为: 65.61535497960534
agent 10 在 53999 次迭代中的资源申请量为: 54.20605797550309
agent 11 在 53999 次迭代中的资源申请量为: 65.42367673401846
agent 12 在 53999 次迭代中的资源申请量为: 38.39518164022969
agent 13 在 53999 次迭代中的资源申请量为: 49.68324623138798
agent 14 在 53999 次迭代中的资源申请量为: 41.348196076456006
agent 15 在 53999 次迭代中的资源申请量为: 42.474235318132436
agent 16 在 53999 次迭代中的资源申请量为: 49.22913190813551
agent 17 在 53999 次迭代中的资源申请量为: 50.09875746179732
agent 18 在 53999 次迭代中的资源申请量为: 46.52237030270356
agent 19 在 53999 次迭代中的资源申请量为: 44.76189093882359
agent 20 在 53999 次迭代中的资源申请量为: 50.780260671816166
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.17570862505862
agent 2 在 55999 次迭代中的资源申请量为: 56.00952657376532
agent 3 在 55999 次迭代中的资源申请量为: 60.62197622352962
agent 4 在 55999 次迭代中的资源申请量为: 37.916432632991466
agent 5 在 55999 次迭代中的资源申请量为: 57.54180594379163
agent 6 在 55999 次迭代中的资源申请量为: 55.318443766067254
agent 7 在 55999 次迭代中的资源申请量为: 65.59675045319692
agent 8 在 55999 次迭代中的资源申请量为: 46.315116159398904
agent 9 在 55999 次迭代中的资源申请量为: 65.61535684345327
agent 10 在 55999 次迭代中的资源申请量为: 54.20609819414635
agent 11 在 55999 次迭代中的资源申请量为: 65.4236804785366
agent 12 在 55999 次迭代中的资源申请量为: 38.39516555439683
agent 13 在 55999 次迭代中的资源申请量为: 49.68323457142065
agent 14 在 55999 次迭代中的资源申请量为: 41.348200718207366
agent 15 在 55999 次迭代中的资源申请量为: 42.47422541758093
agent 16 在 55999 次迭代中的资源申请量为: 49.2291185422582
agent 17 在 55999 次迭代中的资源申请量为: 50.098756479175165
agent 18 在 55999 次迭代中的资源申请量为: 46.52235624530924
agent 19 在 55999 次迭代中的资源申请量为: 44.76186789977312
agent 20 在 55999 次迭代中的资源申请量为: 50.78027095558036
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.17570994149833
agent 2 在 57999 次迭代中的资源申请量为: 56.00954979149891
agent 3 在 57999 次迭代中的资源申请量为: 60.621980022414895
agent 4 在 57999 次迭代中的资源申请量为: 37.91643795038843
agent 5 在 57999 次迭代中的资源申请量为: 57.54181260338083
agent 6 在 57999 次迭代中的资源申请量为: 55.318448702183005
agent 7 在 57999 次迭代中的资源申请量为: 65.59674963314488
agent 8 在 57999 次迭代中的资源申请量为: 46.315106221874295
agent 9 在 57999 次迭代中的资源申请量为: 65.61535863621408
agent 10 在 57999 次迭代中的资源申请量为: 54.206136863601046
agent 11 在 57999 次迭代中的资源申请量为: 65.42368407949911
agent 12 在 57999 次迭代中的资源申请量为: 38.39515008877206
agent 13 在 57999 次迭代中的资源申请量为: 49.68322336124829
agent 14 在 57999 次迭代中的资源申请量为: 41.348205181482314
agent 15 在 57999 次迭代中的资源申请量为: 42.47421589894293
agent 16 在 57999 次迭代中的资源申请量为: 49.22910569185577
agent 17 在 57999 次迭代中的资源申请量为: 50.09875553490225
agent 18 在 57999 次迭代中的资源申请量为: 46.52234273004378
agent 19 在 57999 次迭代中的资源申请量为: 44.76184574892875
agent 20 在 57999 次迭代中的资源申请量为: 50.78028084355205
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.175711209436855
agent 2 在 59999 次迭代中的资源申请量为: 56.00957214537273
agent 3 在 59999 次迭代中的资源申请量为: 60.621983680512606
agent 4 在 59999 次迭代中的资源申请量为: 37.916443070137106
agent 5 在 59999 次迭代中的资源申请量为: 57.54181901560086
agent 6 在 59999 次迭代中的资源申请量为: 55.31845345496318
agent 7 在 59999 次迭代中的资源申请量为: 65.59674884420761
agent 8 在 59999 次迭代中的资源申请量为: 46.315096654699886
agent 9 在 59999 次迭代中的资源申请量为: 65.61536036289394
agent 10 在 59999 次迭代中的资源申请量为: 54.20617409395096
agent 11 在 59999 次迭代中的资源申请量为: 65.42368754706307
agent 12 在 59999 次迭代中的资源申请量为: 38.39513519924404
agent 13 在 59999 次迭代中的资源申请量为: 49.68321256886444
agent 14 在 59999 次迭代中的资源申请量为: 41.3482094789411
agent 15 在 59999 次迭代中的资源申请量为: 42.474206735043005
agent 16 在 59999 次迭代中的资源申请量为: 49.229093320255615
agent 17 在 59999 次迭代中的资源申请量为: 50.0987546262199
agent 18 在 59999 次迭代中的资源申请量为: 46.5223297183404
agent 19 在 59999 次迭代中的资源申请量为: 44.76182442312431
agent 20 在 59999 次迭代中的资源申请量为: 50.78029036383285
agent 21 在 59999 次迭代中的资源申请量为: 6

In [6]:
np.savetxt('./data/01_IOS_oracle_agent_resource_quantity_min_attack.txt',agent_resource_quantity_robust1,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_oracle_average_cost_honest_min_attack.txt',average_cost_honest_robust1,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_oracle_average_resource_honest_min_attack.txt',average_resource_honest_robust1,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_oracle_dual_variable_min_attack.txt',dual_variable_robust1,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+大值攻击

In [7]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [8]:
#定义变量 
agent_resource_quantity_robust2 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust2=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust2 = 0.6
dual_variable_robust2 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust2= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust2=拜占庭用户数量
iteration_robust2=[]#绘图横坐标
A_robust2 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵
neighbor_robust2_help=[]
q_robust2=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust2=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust2=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust2=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
check_robust2=np.zeros(shape=(100,60000))

In [9]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    dual_variable_robust2[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust2*100)),100):
    dual_variable_robust2[i][0]=-0.01
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust2*100)):
    agent_resource_quantity_robust2[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust2=0
for i in range (0,100-int(proportion_robust2*100)):
    sum_robust2 = sum_robust2 + agent_resource_quantity_robust2[i][0]
average_resource_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

#诚实用户平均成本初始化
sum_robust2=0
for i in range(0,100-int(proportion_robust2*100)):
    sum_robust2=sum_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][0]-b_robust2[i])*(agent_resource_quantity_robust2[i][0]-b_robust2[i])
average_cost_honest_robust2[0][0]=sum_robust2/(100-int(proportion_robust2*100))

iteration_robust2.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust2[i][j]!=0:
            neighbor_robust2_help.append(j)
neighbor_robust2=np.array(neighbor_robust2_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust2_help=0
    for j in range(0,15):
        if neighbor_robust2[i][j]==94 or neighbor_robust2[i][j]==95 or neighbor_robust2[i][j]==96 or neighbor_robust2[i][j]==97 or neighbor_robust2[i][j]==98 or neighbor_robust2[i][j]==99:
            q_robust2_help=q_robust2_help+1
    q_robust2.append(q_robust2_help) 

In [10]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,q_robust2):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+大值拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust2[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust2[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust2[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust2[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust2)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust2.append(k)
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust2=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust2*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1]-iteration_update_robust2*((50/100)-(1/100)*agent_resource_quantity_robust2[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust2*100)),100):
            dual_variable_intermediate_robust2.append(dual_variable_robust2[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust2*100)):
            u_neighbor_help_robust2=neighbor_robust2[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust2.append(i)
            for q in range(0, int(q_robust2[i])):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust2=0
                h_robust2=0
                for j in range(0,len(u_neighbor_help_robust2)):#计算当前循环中的h值
                    h_robust2=h_robust2+W_robust2[i][u_neighbor_help_robust2[j]]
                for j in range(0,len(u_neighbor_help_robust2)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust2=dual_variable_weight_average_robust2+(1/h_robust2)*(W_robust2[i][u_neighbor_help_robust2[j]]*dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust2.remove(i)
                neighbor_index_robust2=u_neighbor_help_robust2[0]
                max_distance_roubust2=abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[0]]-dual_variable_weight_average_robust2)
                for j in range(1,len(u_neighbor_help_robust2)):
                    if abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]]-dual_variable_weight_average_robust2)>=max_distance_roubust2:
                        max_distance_roubust2=abs(dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]]-dual_variable_weight_average_robust2)
                        neighbor_index_robust2=u_neighbor_help_robust2[j]
                u_neighbor_help_robust2.remove(neighbor_index_robust2)
                u_neighbor_help_robust2.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust2=0
            h_robust2=0
            for j in range(0,len(u_neighbor_help_robust2)):
                h_robust2=h_robust2+W_robust2[i][u_neighbor_help_robust2[j]]
            for j in range(0,len(u_neighbor_help_robust2)):
                dual_variable_weight_average_robust2=dual_variable_weight_average_robust2+(1/h_robust2)*(W_robust2[i][u_neighbor_help_robust2[j]]*dual_variable_intermediate_robust2[u_neighbor_help_robust2[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust2[i][k]=dual_variable_weight_average_robust2
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust2)):
                if u_neighbor_help_robust2[j]==94 or u_neighbor_help_robust2[j]==95 or u_neighbor_help_robust2[j]==96 or u_neighbor_help_robust2[j]==97 or u_neighbor_help_robust2[j]==98 or u_neighbor_help_robust2[j]==99:
                    check_robust2[i][k]=1        
        
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            dual_variable_robust2[i][k]=dual_variable_robust2[i][k-1]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust2*100)):
            agent_resource_quantity_robust2[i][k]=(dual_variable_robust2[i][k]-2*a_robust2[i]*b_robust2[i])/(-2*a_robust2[i])
            #原变量的投影操作
            if agent_resource_quantity_robust2[i][k] < 0:
                agent_resource_quantity_robust2[i][k]=0
            elif agent_resource_quantity_robust2[i][k] > 100: 
                agent_resource_quantity_robust2[i][k]=100   
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust2*100),100):
            agent_resource_quantity_robust2[i][k]=agent_resource_quantity_robust2[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust2=0
        for i in range (0,100-int(proportion_robust2*100)):
            sum1_robust2 = sum1_robust2 + agent_resource_quantity_robust2[i][k]
        average_resource_honest_robust2[0][k]=sum1_robust2/(100-int(proportion_robust2*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust2=0
        for i in range(0,100-int(proportion_robust2*100)):
            sum2_robust2=sum2_robust2+a_robust2[i]*(agent_resource_quantity_robust2[i][k]-b_robust2[i])*(agent_resource_quantity_robust2[i][k]-b_robust2[i])
        average_cost_honest_robust2[0][k]=sum2_robust2/(100-int(proportion_robust2*100))
        
       #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust2[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust2[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust2[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust2[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust2)
            print("****************************************************************************")
        
        iteration_update_robust2=1/(pow(k+1,0.1)) #递减步长
    
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [11]:
Robust_DGD_IOS_robust2(a_robust2,b_robust2,agent_resource_quantity_robust2,average_resource_honest_robust2,average_cost_honest_robust2,iteration_update_robust2,dual_variable_robust2,A_robust2,iteration_robust2,q_robust2)

****************************DGD算法+IOS+大值拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.12070683276905
agent 2 在 3999 次迭代中的资源申请量为: 55.95304928256159
agent 3 在 3999 次迭代中的资源申请量为: 60.561504445775654
agent 4 在 3999 次迭代中的资源申请量为: 37.87880941536425
agent 5 在 3999 次迭代中的资源申请量为: 57.485123743591
agent 6 在 3999 次迭代中的资源申请量为: 55.264203791618655
agent 7 在 3999 次迭代中的资源申请量为: 65.53125226251791
agent 8 在 3999 次迭代中的资源申请量为: 46.27086551757551
agent 9 在 3999 次迭代中的资源申请量为: 65.55042218484795
agent 10 在 3999 次迭代中的资源申请量为: 54.14963600043463
agent 11 在 3999 次迭代中的资源申请量为: 65.3573429729549
agent 12 在 3999 次迭代中的资源申请量为: 38.35910883660039
agent 13 在 3999 次迭代中的资源申请量为: 49.63453967492609
agent 14 在 3999 次迭代中的资源申请量为: 41.30791885082795
agent 15 在 3999 次迭代中的资源申请量为: 42.43389362994332
agent 16 在 3999 次迭代中的资源申请量为: 49.18225461624181
agent 17 在 3999 次迭代中的资源申请量为: 50.050423332647604
agent 18 在 3999 次迭代中的资源申请量为: 46.47809367381057
agent 19 在 3999 次迭代中的资源申请量为: 44.719362884828016
agent 20 在 3999 次迭代中的资源申请量为: 50.73037511281795
agent 21 在 3999 次迭代中的资源申请量为: 69.8324590448429
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.17297628108825
agent 2 在 5999 次迭代中的资源申请量为: 56.005249429242454
agent 3 在 5999 次迭代中的资源申请量为: 60.61881408119263
agent 4 在 5999 次迭代中的资源申请量为: 37.91426629347785
agent 5 在 5999 次迭代中的资源申请量为: 57.53863333527171
agent 6 在 5999 次迭代中的资源申请量为: 55.315504287381664
agent 7 在 5999 次迭代中的资源申请量为: 65.59365730930715
agent 8 在 5999 次迭代中的资源申请量为: 46.31365787605212
agent 9 在 5999 次迭代中的资源申请量为: 65.61211513602582
agent 10 在 5999 次迭代中的资源申请量为: 54.200781961994586
agent 11 在 5999 次迭代中的资源申请量为: 65.4202496312116
agent 12 在 5999 次迭代中的资源申请量为: 38.39447317853655
agent 13 在 5999 次迭代中的资源申请量为: 49.68164834947196
agent 14 在 5999 次迭代中的资源申请量为: 41.34596408257298
agent 15 在 5999 次迭代中的资源申请量为: 42.472927353235804
agent 16 在 5999 次迭代中的资源申请量为: 49.22773063371897
agent 17 在 5999 次迭代中的资源申请量为: 50.09649678214453
agent 18 在 5999 次迭代中的资源申请量为: 46.52113818982445
agent 19 在 5999 次迭代中的资源申请量为: 44.76131545941006
agent 20 在 5999 次迭代中的资源申请量为: 50.77720713463902
agent 21 在 5999 次迭代中的资源申请量为: 69.89814844393362
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.175489324357414
agent 2 在 7999 次迭代中的资源申请量为: 56.007965353906926
agent 3 在 7999 次迭代中的资源申请量为: 60.62159158212441
agent 4 在 7999 次迭代中的资源申请量为: 37.91601268471958
agent 5 在 7999 次迭代中的资源申请量为: 57.541255995392
agent 6 在 7999 次迭代中的资源申请量为: 55.31800509270703
agent 7 在 7999 次迭代中的资源申请量为: 65.5966351524103
agent 8 在 7999 次迭代中的资源申请量为: 46.31561158193708
agent 9 在 7999 次迭代中的资源申请量为: 65.61508346873212
agent 10 在 7999 次迭代中的资源申请量为: 54.20359292127057
agent 11 在 7999 次迭代中的资源申请量为: 65.4232930589376
agent 12 在 7999 次迭代中的资源申请量为: 38.39601946339723
agent 13 在 7999 次迭代中的资源申请量为: 49.68379656357857
agent 14 在 7999 次迭代中的资源申请量为: 41.347826259941925
agent 15 在 7999 次迭代中的资源申请量为: 42.47470517393577
agent 16 在 7999 次迭代中的资源申请量为: 49.229785314883415
agent 17 在 7999 次迭代中的资源申请量为: 50.09869212325308
agent 18 在 7999 次迭代中的资源申请量为: 46.523070267456085
agent 19 在 7999 次迭代中的资源申请量为: 44.76311399652334
agent 20 在 7999 次迭代中的资源申请量为: 50.77954075956235
agent 21 在 7999 次迭代中的资源申请量为: 69.90139441831356
agent 2

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.17563138506366
agent 2 在 9999 次迭代中的资源申请量为: 56.00827401335625
agent 3 在 9999 次迭代中的资源申请量为: 60.62176524552095
agent 4 在 9999 次迭代中的资源申请量为: 37.91614275552205
agent 5 在 9999 次迭代中的资源申请量为: 57.541441861572025
agent 6 在 9999 次迭代中的资源申请量为: 55.318172297737725
agent 7 在 9999 次迭代中的资源申请量为: 65.59678655677419
agent 8 在 9999 次迭代中的资源申请量为: 46.31564401731814
agent 9 在 9999 次迭代中的资源申请量为: 65.61525293818953
agent 10 在 9999 次迭代中的资源申请量为: 54.204016573454865
agent 11 在 9999 次迭代中的资源申请量为: 65.42347936737477
agent 12 在 9999 次迭代中的资源申请量为: 38.39599103420858
agent 13 在 9999 次迭代中的资源申请量为: 49.683830204976175
agent 14 在 9999 次迭代中的资源申请量为: 41.347956362274005
agent 15 在 9999 次迭代中的资源申请量为: 42.47473129706811
agent 16 在 9999 次迭代中的资源申请量为: 49.229802355617565
agent 17 在 9999 次迭代中的资源申请量为: 50.09880132319444
agent 18 在 9999 次迭代中的资源申请量为: 46.52307609052926
agent 19 在 9999 次迭代中的资源申请量为: 44.76305130458887
agent 20 在 9999 次迭代中的资源申请量为: 50.77973436797195
agent 21 在 9999 次迭代中的资源申请量为: 69.90164390319808
ag

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.17570583156672
agent 2 在 51999 次迭代中的资源申请量为: 56.00947727472043
agent 3 在 51999 次迭代中的资源申请量为: 60.621968159243025
agent 4 在 51999 次迭代中的资源申请量为: 37.916421343091805
agent 5 在 51999 次迭代中的资源申请量为: 57.54179180475173
agent 6 在 51999 次迭代中的资源申请量为: 55.318433286222785
agent 7 在 51999 次迭代中的资源申请量为: 65.59675219664979
agent 8 在 51999 次迭代中的资源申请量为: 46.31513726231826
agent 9 在 51999 次迭代中的资源申请量为: 65.61535303908309
agent 10 在 51999 次迭代中的资源申请量为: 54.20601608468502
agent 11 在 51999 次迭代中的资源申请量为: 65.42367283460275
agent 12 在 51999 次迭代中的资源申请量为: 38.39519839555732
agent 13 在 51999 次迭代中的资源申请量为: 49.683258376913756
agent 14 在 51999 次迭代中的资源申请量为: 41.34819124208708
agent 15 在 51999 次迭代中的资源申请量为: 42.474245630962805
agent 16 在 51999 次迭代中的资源申请量为: 49.229145830464255
agent 17 在 51999 次迭代中的资源申请量为: 50.09875848585519
agent 18 在 51999 次迭代中的资源申请量为: 46.522384945319395
agent 19 在 51999 次迭代中的资源申请量为: 44.76191493665629
agent 20 在 51999 次迭代中的资源申请量为: 50.78024996086717
agent 21 在 51999 次迭代中的资源申请量

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.17570725644687
agent 2 在 53999 次迭代中的资源申请量为: 56.00950242610763
agent 3 在 53999 次迭代中的资源申请量为: 60.62197227313143
agent 4 在 53999 次迭代中的资源申请量为: 37.91642710284778
agent 5 在 53999 次迭代中的资源申请量为: 57.54179901794708
agent 6 在 53999 次迭代中的资源申请量为: 55.31843863262337
agent 7 在 53999 次迭代中的资源申请量为: 65.59675130679224
agent 8 在 53999 次迭代中的资源申请量为: 46.31512649564598
agent 9 在 53999 次迭代中的资源申请量为: 65.61535497960534
agent 10 在 53999 次迭代中的资源申请量为: 54.20605797550309
agent 11 在 53999 次迭代中的资源申请量为: 65.42367673401846
agent 12 在 53999 次迭代中的资源申请量为: 38.39518164022969
agent 13 在 53999 次迭代中的资源申请量为: 49.68324623138798
agent 14 在 53999 次迭代中的资源申请量为: 41.348196076456006
agent 15 在 53999 次迭代中的资源申请量为: 42.474235318132436
agent 16 在 53999 次迭代中的资源申请量为: 49.22913190813551
agent 17 在 53999 次迭代中的资源申请量为: 50.09875746179732
agent 18 在 53999 次迭代中的资源申请量为: 46.52237030270356
agent 19 在 53999 次迭代中的资源申请量为: 44.76189093882359
agent 20 在 53999 次迭代中的资源申请量为: 50.780260671816166
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.17570862505862
agent 2 在 55999 次迭代中的资源申请量为: 56.00952657376532
agent 3 在 55999 次迭代中的资源申请量为: 60.62197622352962
agent 4 在 55999 次迭代中的资源申请量为: 37.916432632991466
agent 5 在 55999 次迭代中的资源申请量为: 57.54180594379163
agent 6 在 55999 次迭代中的资源申请量为: 55.318443766067254
agent 7 在 55999 次迭代中的资源申请量为: 65.59675045319692
agent 8 在 55999 次迭代中的资源申请量为: 46.315116159398904
agent 9 在 55999 次迭代中的资源申请量为: 65.61535684345327
agent 10 在 55999 次迭代中的资源申请量为: 54.20609819414635
agent 11 在 55999 次迭代中的资源申请量为: 65.4236804785366
agent 12 在 55999 次迭代中的资源申请量为: 38.39516555439683
agent 13 在 55999 次迭代中的资源申请量为: 49.68323457142065
agent 14 在 55999 次迭代中的资源申请量为: 41.348200718207366
agent 15 在 55999 次迭代中的资源申请量为: 42.47422541758093
agent 16 在 55999 次迭代中的资源申请量为: 49.2291185422582
agent 17 在 55999 次迭代中的资源申请量为: 50.098756479175165
agent 18 在 55999 次迭代中的资源申请量为: 46.52235624530924
agent 19 在 55999 次迭代中的资源申请量为: 44.76186789977312
agent 20 在 55999 次迭代中的资源申请量为: 50.78027095558036
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.17570994149833
agent 2 在 57999 次迭代中的资源申请量为: 56.00954979149891
agent 3 在 57999 次迭代中的资源申请量为: 60.621980022414895
agent 4 在 57999 次迭代中的资源申请量为: 37.91643795038843
agent 5 在 57999 次迭代中的资源申请量为: 57.54181260338083
agent 6 在 57999 次迭代中的资源申请量为: 55.318448702183005
agent 7 在 57999 次迭代中的资源申请量为: 65.59674963314488
agent 8 在 57999 次迭代中的资源申请量为: 46.315106221874295
agent 9 在 57999 次迭代中的资源申请量为: 65.61535863621408
agent 10 在 57999 次迭代中的资源申请量为: 54.206136863601046
agent 11 在 57999 次迭代中的资源申请量为: 65.42368407949911
agent 12 在 57999 次迭代中的资源申请量为: 38.39515008877206
agent 13 在 57999 次迭代中的资源申请量为: 49.68322336124829
agent 14 在 57999 次迭代中的资源申请量为: 41.348205181482314
agent 15 在 57999 次迭代中的资源申请量为: 42.47421589894293
agent 16 在 57999 次迭代中的资源申请量为: 49.22910569185577
agent 17 在 57999 次迭代中的资源申请量为: 50.09875553490225
agent 18 在 57999 次迭代中的资源申请量为: 46.52234273004378
agent 19 在 57999 次迭代中的资源申请量为: 44.76184574892875
agent 20 在 57999 次迭代中的资源申请量为: 50.78028084355205
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.175711209436855
agent 2 在 59999 次迭代中的资源申请量为: 56.00957214537273
agent 3 在 59999 次迭代中的资源申请量为: 60.621983680512606
agent 4 在 59999 次迭代中的资源申请量为: 37.916443070137106
agent 5 在 59999 次迭代中的资源申请量为: 57.54181901560086
agent 6 在 59999 次迭代中的资源申请量为: 55.31845345496318
agent 7 在 59999 次迭代中的资源申请量为: 65.59674884420761
agent 8 在 59999 次迭代中的资源申请量为: 46.315096654699886
agent 9 在 59999 次迭代中的资源申请量为: 65.61536036289394
agent 10 在 59999 次迭代中的资源申请量为: 54.20617409395096
agent 11 在 59999 次迭代中的资源申请量为: 65.42368754706307
agent 12 在 59999 次迭代中的资源申请量为: 38.39513519924404
agent 13 在 59999 次迭代中的资源申请量为: 49.68321256886444
agent 14 在 59999 次迭代中的资源申请量为: 41.3482094789411
agent 15 在 59999 次迭代中的资源申请量为: 42.474206735043005
agent 16 在 59999 次迭代中的资源申请量为: 49.229093320255615
agent 17 在 59999 次迭代中的资源申请量为: 50.0987546262199
agent 18 在 59999 次迭代中的资源申请量为: 46.5223297183404
agent 19 在 59999 次迭代中的资源申请量为: 44.76182442312431
agent 20 在 59999 次迭代中的资源申请量为: 50.78029036383285
agent 21 在 59999 次迭代中的资源申请量为: 6

In [12]:
np.savetxt('./data/01_IOS_oracle_agent_resource_quantity_max_attack.txt',agent_resource_quantity_robust2,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_oracle_average_cost_honest_max_attack.txt',average_cost_honest_robust2,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_oracle_average_resource_honest_max_attack.txt',average_resource_honest_robust2,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_oracle_dual_variable_max_attack.txt',dual_variable_robust2,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+大值高斯攻击1

In [13]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [14]:
#定义变量 
agent_resource_quantity_robust3 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust3=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust3 = 0.6
dual_variable_robust3 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust3= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust3=[]#绘图横坐标
A_robust3 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust3_help=[]
q_robust3=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust3=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust3=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust3=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
random_attack3_robust3= np.loadtxt("./data/random_Gaussian_attack_value_-30_5.txt")
check_robust3=np.zeros(shape=(100,60000))

In [15]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    dual_variable_robust3[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust3*100)),100):
    dual_variable_robust3[i][0]=random_attack3_robust3[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust3*100)):
    agent_resource_quantity_robust3[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust3=0
for i in range (0,100-int(proportion_robust3*100)):
    sum_robust3 = sum_robust3 + agent_resource_quantity_robust3[i][0]
average_resource_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

#诚实用户平均成本初始化
sum_robust3=0
for i in range(0,100-int(proportion_robust3*100)):
    sum_robust3=sum_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][0]-b_robust3[i])*(agent_resource_quantity_robust3[i][0]-b_robust3[i])
average_cost_honest_robust3[0][0]=sum_robust3/(100-int(proportion_robust3*100))

iteration_robust3.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust3[i][j]!=0:
            neighbor_robust3_help.append(j)
neighbor_robust3=np.array(neighbor_robust3_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust3_help=0
    for j in range(0,15):
        if neighbor_robust3[i][j]==94 or neighbor_robust3[i][j]==95 or neighbor_robust3[i][j]==96 or neighbor_robust3[i][j]==97 or neighbor_robust3[i][j]==98 or neighbor_robust3[i][j]==99:
            q_robust3_help=q_robust3_help+1
    q_robust3.append(q_robust3_help) 

In [16]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,q_robust3):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust3[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust3[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust3[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust3[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust3)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust3.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust3=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust3*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1]-iteration_update_robust3*((50/100)-(1/100)*agent_resource_quantity_robust3[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust3*100)),100):
            dual_variable_intermediate_robust3.append(dual_variable_robust3[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust3*100)):
            u_neighbor_help_robust3=neighbor_robust3[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust3.append(i)
            for q in range(0, int(q_robust3[i])):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust3=0
                h_robust3=0
                for j in range(0,len(u_neighbor_help_robust3)):#计算当前循环中的h值
                    h_robust3=h_robust3+W_robust3[i][u_neighbor_help_robust3[j]]
                for j in range(0,len(u_neighbor_help_robust3)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust3=dual_variable_weight_average_robust3+(1/h_robust3)*(W_robust3[i][u_neighbor_help_robust3[j]]*dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust3.remove(i)
                neighbor_index_robust3=u_neighbor_help_robust3[0]
                max_distance_roubust3=abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[0]]-dual_variable_weight_average_robust3)
                for j in range(1,len(u_neighbor_help_robust3)):
                    if abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]]-dual_variable_weight_average_robust3)>=max_distance_roubust3:
                        max_distance_roubust3=abs(dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]]-dual_variable_weight_average_robust3)
                        neighbor_index_robust3=u_neighbor_help_robust3[j]
                u_neighbor_help_robust3.remove(neighbor_index_robust3)
                u_neighbor_help_robust3.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust3=0
            h_robust3=0
            for j in range(0,len(u_neighbor_help_robust3)):
                h_robust3=h_robust3+W_robust3[i][u_neighbor_help_robust3[j]]
            for j in range(0,len(u_neighbor_help_robust3)):
                dual_variable_weight_average_robust3=dual_variable_weight_average_robust3+(1/h_robust3)*(W_robust3[i][u_neighbor_help_robust3[j]]*dual_variable_intermediate_robust3[u_neighbor_help_robust3[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust3[i][k]=dual_variable_weight_average_robust3
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust3)):
                if u_neighbor_help_robust3[j]==94 or u_neighbor_help_robust3[j]==95 or u_neighbor_help_robust3[j]==96 or u_neighbor_help_robust3[j]==97 or u_neighbor_help_robust3[j]==98 or u_neighbor_help_robust3[j]==99:
                    check_robust3[i][k]=1             
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            dual_variable_robust3[i][k]=random_attack3_robust3[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust3*100)):
            agent_resource_quantity_robust3[i][k]=(dual_variable_robust3[i][k]-2*a_robust3[i]*b_robust3[i])/(-2*a_robust3[i])
            #原变量的投影操作
            if agent_resource_quantity_robust3[i][k] < 0:
                agent_resource_quantity_robust3[i][k]=0
            elif agent_resource_quantity_robust3[i][k] > 100: 
                agent_resource_quantity_robust3[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust3*100),100):
            agent_resource_quantity_robust3[i][k]=agent_resource_quantity_robust3[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust3=0
        for i in range (0,100-int(proportion_robust3*100)):
            sum1_robust3 = sum1_robust3 + agent_resource_quantity_robust3[i][k]
        average_resource_honest_robust3[0][k]=sum1_robust3/(100-int(proportion_robust3*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust3=0
        for i in range(0,100-int(proportion_robust3*100)):
            sum2_robust3=sum2_robust3+a_robust3[i]*(agent_resource_quantity_robust3[i][k]-b_robust3[i])*(agent_resource_quantity_robust3[i][k]-b_robust3[i])
        average_cost_honest_robust3[0][k]=sum2_robust3/(100-int(proportion_robust3*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust3[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust3[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust3[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust3[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust3)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust3)
            print("****************************************************************************")'''
        
        iteration_update_robust3=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [17]:
Robust_DGD_IOS_robust3(a_robust3,b_robust3,agent_resource_quantity_robust3,average_resource_honest_robust3,average_cost_honest_robust3,iteration_update_robust3,dual_variable_robust3,A_robust3,iteration_robust3,q_robust3)

****************************DGD算法+IOS+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.12070687660069
agent 2 在 3999 次迭代中的资源申请量为: 55.95304932606379
agent 3 在 3999 次迭代中的资源申请量为: 60.56150449380474
agent 4 在 3999 次迭代中的资源申请量为: 37.87880944504253
agent 5 在 3999 次迭代中的资源申请量为: 57.4851237883968
agent 6 在 3999 次迭代中的资源申请量为: 55.264203834592585
agent 7 在 3999 次迭代中的资源申请量为: 65.5312523148786
agent 8 在 3999 次迭代中的资源申请量为: 46.270865553596394
agent 9 在 3999 次迭代中的资源申请量为: 65.55042223657894
agent 10 在 3999 次迭代中的资源申请量为: 54.14963604286106
agent 11 在 3999 次迭代中的资源申请量为: 65.35734302568166
agent 12 在 3999 次迭代中的资源申请量为: 38.35910886645853
agent 13 在 3999 次迭代中的资源申请量为: 49.63453971458363
agent 14 在 3999 次迭代中的资源申请量为: 41.307918882688135
agent 15 在 3999 次迭代中的资源申请量为: 42.433893662805964
agent 16 在 3999 次迭代中的资源申请量为: 49.18225465455002
agent 17 在 3999 次迭代中的资源申请量为: 50.050423371309535
agent 18 在 3999 次迭代中的资源申请量为: 46.47809371008731
agent 19 在 3999 次迭代中的资源申请量为: 44.71936292029573
agent 20 在 3999 次迭代中的资源申请量为: 50.730375151982024
agent 21 在 3999 次迭代中的资源申请量为: 69.83245909981274
agen

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.17297628319759
agent 2 在 5999 次迭代中的资源申请量为: 56.00524943133593
agent 3 在 5999 次迭代中的资源申请量为: 60.618814083503956
agent 4 在 5999 次迭代中的资源申请量为: 37.91426629490608
agent 5 在 5999 次迭代中的资源申请量为: 57.538633337427925
agent 6 在 5999 次迭代中的资源申请量为: 55.31550428944972
agent 7 在 5999 次迭代中的资源申请量为: 65.59365731182692
agent 8 在 5999 次迭代中的资源申请量为: 46.31365787778555
agent 9 在 5999 次迭代中的资源申请量为: 65.61211513851529
agent 10 在 5999 次迭代中的资源申请量为: 54.2007819640363
agent 11 在 5999 次迭代中的资源申请量为: 65.42024963374901
agent 12 在 5999 次迭代中的资源申请量为: 38.394473179973424
agent 13 在 5999 次迭代中的资源申请量为: 49.68164835138044
agent 14 在 5999 次迭代中的资源申请量为: 41.345964084106214
agent 15 在 5999 次迭代中的资源申请量为: 42.47292735481726
agent 16 在 5999 次迭代中的资源申请量为: 49.227730635562494
agent 17 在 5999 次迭代中的资源申请量为: 50.09649678400507
agent 18 在 5999 次迭代中的资源申请量为: 46.521138191570216
agent 19 在 5999 次迭代中的资源申请量为: 44.76131546111688
agent 20 在 5999 次迭代中的资源申请量为: 50.77720713652374
agent 21 在 5999 次迭代中的资源申请量为: 69.89814844657899
age

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.17548932446978
agent 2 在 7999 次迭代中的资源申请量为: 56.007965354018445
agent 3 在 7999 次迭代中的资源申请量为: 60.62159158224755
agent 4 在 7999 次迭代中的资源申请量为: 37.91601268479565
agent 5 在 7999 次迭代中的资源申请量为: 57.54125599550688
agent 6 在 7999 次迭代中的资源申请量为: 55.3180050928172
agent 7 在 7999 次迭代中的资源申请量为: 65.59663515254458
agent 8 在 7999 次迭代中的资源申请量为: 46.31561158202943
agent 9 在 7999 次迭代中的资源申请量为: 65.61508346886475
agent 10 在 7999 次迭代中的资源申请量为: 54.203592921379325
agent 11 在 7999 次迭代中的资源申请量为: 65.42329305907278
agent 12 在 7999 次迭代中的资源申请量为: 38.396019463473785
agent 13 在 7999 次迭代中的资源申请量为: 49.68379656368025
agent 14 在 7999 次迭代中的资源申请量为: 41.347826260023616
agent 15 在 7999 次迭代中的资源申请量为: 42.47470517402002
agent 16 在 7999 次迭代中的资源申请量为: 49.22978531498162
agent 17 在 7999 次迭代中的资源申请量为: 50.09869212335221
agent 18 在 7999 次迭代中的资源申请量为: 46.523070267549095
agent 19 在 7999 次迭代中的资源申请量为: 44.76311399661427
agent 20 在 7999 次迭代中的资源申请量为: 50.77954075966276
agent 21 在 7999 次迭代中的资源申请量为: 69.9013944184545
agent

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.17563138507008
agent 2 在 9999 次迭代中的资源申请量为: 56.00827401336264
agent 3 在 9999 次迭代中的资源申请量为: 60.62176524552799
agent 4 在 9999 次迭代中的资源申请量为: 37.91614275552641
agent 5 在 9999 次迭代中的资源申请量为: 57.5414418615786
agent 6 在 9999 次迭代中的资源申请量为: 55.31817229774403
agent 7 在 9999 次迭代中的资源申请量为: 65.59678655678188
agent 8 在 9999 次迭代中的资源申请量为: 46.31564401732342
agent 9 在 9999 次迭代中的资源申请量为: 65.61525293819712
agent 10 在 9999 次迭代中的资源申请量为: 54.204016573461075
agent 11 在 9999 次迭代中的资源申请量为: 65.4234793673825
agent 12 在 9999 次迭代中的资源申请量为: 38.395991034212955
agent 13 在 9999 次迭代中的资源申请量为: 49.683830204981994
agent 14 在 9999 次迭代中的资源申请量为: 41.34795636227867
agent 15 在 9999 次迭代中的资源申请量为: 42.47473129707293
agent 16 在 9999 次迭代中的资源申请量为: 49.22980235562317
agent 17 在 9999 次迭代中的资源申请量为: 50.09880132320011
agent 18 在 9999 次迭代中的资源申请量为: 46.52307609053457
agent 19 在 9999 次迭代中的资源申请量为: 44.76305130459407
agent 20 在 9999 次迭代中的资源申请量为: 50.779734367977696
agent 21 在 9999 次迭代中的资源申请量为: 69.90164390320612
agent 

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.17570583156672
agent 2 在 51999 次迭代中的资源申请量为: 56.00947727472043
agent 3 在 51999 次迭代中的资源申请量为: 60.621968159243025
agent 4 在 51999 次迭代中的资源申请量为: 37.916421343091805
agent 5 在 51999 次迭代中的资源申请量为: 57.54179180475173
agent 6 在 51999 次迭代中的资源申请量为: 55.318433286222785
agent 7 在 51999 次迭代中的资源申请量为: 65.59675219664979
agent 8 在 51999 次迭代中的资源申请量为: 46.31513726231826
agent 9 在 51999 次迭代中的资源申请量为: 65.61535303908309
agent 10 在 51999 次迭代中的资源申请量为: 54.20601608468502
agent 11 在 51999 次迭代中的资源申请量为: 65.42367283460275
agent 12 在 51999 次迭代中的资源申请量为: 38.39519839555732
agent 13 在 51999 次迭代中的资源申请量为: 49.683258376913756
agent 14 在 51999 次迭代中的资源申请量为: 41.34819124208708
agent 15 在 51999 次迭代中的资源申请量为: 42.474245630962805
agent 16 在 51999 次迭代中的资源申请量为: 49.229145830464255
agent 17 在 51999 次迭代中的资源申请量为: 50.09875848585519
agent 18 在 51999 次迭代中的资源申请量为: 46.522384945319395
agent 19 在 51999 次迭代中的资源申请量为: 44.76191493665629
agent 20 在 51999 次迭代中的资源申请量为: 50.78024996086717
agent 21 在 51999 次迭代中的资源申请量

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.17570725644687
agent 2 在 53999 次迭代中的资源申请量为: 56.00950242610763
agent 3 在 53999 次迭代中的资源申请量为: 60.62197227313143
agent 4 在 53999 次迭代中的资源申请量为: 37.91642710284778
agent 5 在 53999 次迭代中的资源申请量为: 57.54179901794708
agent 6 在 53999 次迭代中的资源申请量为: 55.31843863262337
agent 7 在 53999 次迭代中的资源申请量为: 65.59675130679224
agent 8 在 53999 次迭代中的资源申请量为: 46.31512649564598
agent 9 在 53999 次迭代中的资源申请量为: 65.61535497960534
agent 10 在 53999 次迭代中的资源申请量为: 54.20605797550309
agent 11 在 53999 次迭代中的资源申请量为: 65.42367673401846
agent 12 在 53999 次迭代中的资源申请量为: 38.39518164022969
agent 13 在 53999 次迭代中的资源申请量为: 49.68324623138798
agent 14 在 53999 次迭代中的资源申请量为: 41.348196076456006
agent 15 在 53999 次迭代中的资源申请量为: 42.474235318132436
agent 16 在 53999 次迭代中的资源申请量为: 49.22913190813551
agent 17 在 53999 次迭代中的资源申请量为: 50.09875746179732
agent 18 在 53999 次迭代中的资源申请量为: 46.52237030270356
agent 19 在 53999 次迭代中的资源申请量为: 44.76189093882359
agent 20 在 53999 次迭代中的资源申请量为: 50.780260671816166
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.17570862505862
agent 2 在 55999 次迭代中的资源申请量为: 56.00952657376532
agent 3 在 55999 次迭代中的资源申请量为: 60.62197622352962
agent 4 在 55999 次迭代中的资源申请量为: 37.916432632991466
agent 5 在 55999 次迭代中的资源申请量为: 57.54180594379163
agent 6 在 55999 次迭代中的资源申请量为: 55.318443766067254
agent 7 在 55999 次迭代中的资源申请量为: 65.59675045319692
agent 8 在 55999 次迭代中的资源申请量为: 46.315116159398904
agent 9 在 55999 次迭代中的资源申请量为: 65.61535684345327
agent 10 在 55999 次迭代中的资源申请量为: 54.20609819414635
agent 11 在 55999 次迭代中的资源申请量为: 65.4236804785366
agent 12 在 55999 次迭代中的资源申请量为: 38.39516555439683
agent 13 在 55999 次迭代中的资源申请量为: 49.68323457142065
agent 14 在 55999 次迭代中的资源申请量为: 41.348200718207366
agent 15 在 55999 次迭代中的资源申请量为: 42.47422541758093
agent 16 在 55999 次迭代中的资源申请量为: 49.2291185422582
agent 17 在 55999 次迭代中的资源申请量为: 50.098756479175165
agent 18 在 55999 次迭代中的资源申请量为: 46.52235624530924
agent 19 在 55999 次迭代中的资源申请量为: 44.76186789977312
agent 20 在 55999 次迭代中的资源申请量为: 50.78027095558036
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.17570994149833
agent 2 在 57999 次迭代中的资源申请量为: 56.00954979149891
agent 3 在 57999 次迭代中的资源申请量为: 60.621980022414895
agent 4 在 57999 次迭代中的资源申请量为: 37.91643795038843
agent 5 在 57999 次迭代中的资源申请量为: 57.54181260338083
agent 6 在 57999 次迭代中的资源申请量为: 55.318448702183005
agent 7 在 57999 次迭代中的资源申请量为: 65.59674963314488
agent 8 在 57999 次迭代中的资源申请量为: 46.315106221874295
agent 9 在 57999 次迭代中的资源申请量为: 65.61535863621408
agent 10 在 57999 次迭代中的资源申请量为: 54.206136863601046
agent 11 在 57999 次迭代中的资源申请量为: 65.42368407949911
agent 12 在 57999 次迭代中的资源申请量为: 38.39515008877206
agent 13 在 57999 次迭代中的资源申请量为: 49.68322336124829
agent 14 在 57999 次迭代中的资源申请量为: 41.348205181482314
agent 15 在 57999 次迭代中的资源申请量为: 42.47421589894293
agent 16 在 57999 次迭代中的资源申请量为: 49.22910569185577
agent 17 在 57999 次迭代中的资源申请量为: 50.09875553490225
agent 18 在 57999 次迭代中的资源申请量为: 46.52234273004378
agent 19 在 57999 次迭代中的资源申请量为: 44.76184574892875
agent 20 在 57999 次迭代中的资源申请量为: 50.78028084355205
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.175711209436855
agent 2 在 59999 次迭代中的资源申请量为: 56.00957214537273
agent 3 在 59999 次迭代中的资源申请量为: 60.621983680512606
agent 4 在 59999 次迭代中的资源申请量为: 37.916443070137106
agent 5 在 59999 次迭代中的资源申请量为: 57.54181901560086
agent 6 在 59999 次迭代中的资源申请量为: 55.31845345496318
agent 7 在 59999 次迭代中的资源申请量为: 65.59674884420761
agent 8 在 59999 次迭代中的资源申请量为: 46.315096654699886
agent 9 在 59999 次迭代中的资源申请量为: 65.61536036289394
agent 10 在 59999 次迭代中的资源申请量为: 54.20617409395096
agent 11 在 59999 次迭代中的资源申请量为: 65.42368754706307
agent 12 在 59999 次迭代中的资源申请量为: 38.39513519924404
agent 13 在 59999 次迭代中的资源申请量为: 49.68321256886444
agent 14 在 59999 次迭代中的资源申请量为: 41.3482094789411
agent 15 在 59999 次迭代中的资源申请量为: 42.474206735043005
agent 16 在 59999 次迭代中的资源申请量为: 49.229093320255615
agent 17 在 59999 次迭代中的资源申请量为: 50.0987546262199
agent 18 在 59999 次迭代中的资源申请量为: 46.5223297183404
agent 19 在 59999 次迭代中的资源申请量为: 44.76182442312431
agent 20 在 59999 次迭代中的资源申请量为: 50.78029036383285
agent 21 在 59999 次迭代中的资源申请量为: 6

In [18]:
np.savetxt('./data/01_IOS_oracle_agent_resource_quantity_Gaussian_attack_large.txt',agent_resource_quantity_robust3,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_oracle_average_cost_honest_Gaussian_attack_large.txt',average_cost_honest_robust3,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_oracle_average_resource_honest_Gaussian_attack_large.txt',average_resource_honest_robust3,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_oracle_dual_variable_Gaussian_attack_large.txt',dual_variable_robust3,fmt='%0.8f')#用户对偶变量

# DGD+IOS算法+小值高斯攻击2

In [19]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [20]:
#定义变量 
agent_resource_quantity_robust4 = np.zeros(shape=(100,60000)) #100个agent的资源量 定义成一个矩阵 100*2000 
average_resource_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户资源量的平均值
average_cost_honest_robust4=np.zeros(shape=(1,60000))#存放每一次迭代后诚实用户成本的平均值
iteration_update_robust4 = 0.6
dual_variable_robust4 = np.zeros(shape=(100,60000))#对偶变量
proportion_robust4= 0.06 #100个用户中拜占庭用户的比例 100*proportion_robust1=拜占庭用户数量
iteration_robust4=[]#绘图横坐标
A_robust4 = np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt")#对称双随机矩阵 对角元素为0
neighbor_robust4_help=[]
q_robust4=[] #IOS鲁棒聚合中丢弃的数据数量 每个用户丢弃的数据数量等于该用户邻居中拜占庭用户的数量
W_robust4=np.loadtxt("100_Network_Topology_Adjacency_Matrix_B_0.0625(the value of bii=0.0625).txt") #双随机的网络拓扑矩阵W（对角线处非零）
a_robust4=np.loadtxt("100_uniform_distribution_(1,2).txt")#每个agent的成本函数中的超参数
b_robust4=np.loadtxt("100_normal_distribution_(2,0.6).txt")#每个agent的成本函数中的超参数
random_attack3_robust4= np.loadtxt("./data/random_Gaussian_attack_value_-300_40.txt")
check_robust4=np.zeros(shape=(100,60000))

In [21]:
#诚实用户对偶变量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    dual_variable_robust4[i][0]=0
    
#拜占庭用户对偶变量的初始化
for i in range (100-int((proportion_robust4*100)),100):
    dual_variable_robust4[i][0]=random_attack3_robust4[i-94][0]
    
#诚实用户资源量的初始化
for i in range (0,100-int(proportion_robust4*100)):
    agent_resource_quantity_robust4[i][0]=0
    
#诚实用户的平均资源量初始化
sum_robust4=0
for i in range (0,100-int(proportion_robust4*100)):
    sum_robust4 = sum_robust4 + agent_resource_quantity_robust4[i][0]
average_resource_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

#诚实用户平均成本初始化
sum_robust4=0
for i in range(0,100-int(proportion_robust4*100)):
    sum_robust4=sum_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][0]-b_robust4[i])*(agent_resource_quantity_robust4[i][0]-b_robust4[i])
average_cost_honest_robust4[0][0]=sum_robust4/(100-int(proportion_robust4*100))

iteration_robust4.append(0)

#对每个用户的15个邻居进行初始化
for i in range (0,100):
    for j in range (0,100):
        if A_robust4[i][j]!=0:
            neighbor_robust4_help.append(j)
neighbor_robust4=np.array(neighbor_robust4_help).reshape(100,15)

#计算诚实用户的拜占庭邻居用户数量
for i in range(0,100):
    q_robust4_help=0
    for j in range(0,15):
        if neighbor_robust4[i][j]==94 or neighbor_robust4[i][j]==95 or neighbor_robust4[i][j]==96 or neighbor_robust4[i][j]==97 or neighbor_robust4[i][j]==98 or neighbor_robust4[i][j]==99:
            q_robust4_help=q_robust4_help+1
    q_robust4.append(q_robust4_help)

In [22]:
#DGD+IOS算法 
def Robust_DGD_IOS_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,q_robust4):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************DGD算法+IOS+高斯拜占庭攻击****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity_robust4[i][0])
    print("*************************")
    print("初始诚实用户资源申请量平均值avg为:",average_resource_honest_robust4[0][0])
    print("*************************")
    print("初始诚实用户平均成本为:",average_cost_honest_robust4[0][0])
    print("*************************")
    for i in range(0,100):
        print("agent",(i+1),"的初始对偶变量为:",dual_variable_robust4[i][0])
    print("*************************")
    print("迭代步长为:",iteration_update_robust4)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,60000):#迭代次数
        iteration_robust4.append(k)        
        
        #诚实用户对偶变量的迭代 
        #阶段一：梯度下降
        dual_variable_intermediate_robust4=[] #存放当前迭代中100个用户对对偶变量做完梯度下降后得到的中间值
        #计算诚实用户梯度下降后的对偶变量中间值
        for i in range(0,100-int(proportion_robust4*100)):
            #每个用户对对偶变量先进行一次梯度下降
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1]-iteration_update_robust4*((50/100)-(1/100)*agent_resource_quantity_robust4[i][k-1]))       
        #计算拜占庭用户梯度下降后的对偶变量中间值
        for i in range (100-int((proportion_robust4*100)),100):
            dual_variable_intermediate_robust4.append(dual_variable_robust4[i][k-1])
        #阶段二：IOS鲁棒聚合 每个用户聚合自身以及邻居的对偶值    
        for i in range (0,100-int(proportion_robust4*100)):
            u_neighbor_help_robust4=neighbor_robust4[i].tolist()#将用户i的邻居序号存在辅助变量u中
            u_neighbor_help_robust4.append(i)
            for q in range(0, int(q_robust4[i])):#剔除q个邻居的信息
                #计算当前的h-权重
                dual_variable_weight_average_robust4=0
                h_robust4=0
                for j in range(0,len(u_neighbor_help_robust4)):#计算当前循环中的h值
                    h_robust4=h_robust4+W_robust4[i][u_neighbor_help_robust4[j]]
                for j in range(0,len(u_neighbor_help_robust4)):#计算当前循环中的对偶权重平均值
                    dual_variable_weight_average_robust4=dual_variable_weight_average_robust4+(1/h_robust4)*(W_robust4[i][u_neighbor_help_robust4[j]]*dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]])
                #剔除一个对偶值离权重平均对偶值最远的用户 保留自身的值
                u_neighbor_help_robust4.remove(i)
                neighbor_index_robust4=u_neighbor_help_robust4[0]
                max_distance_roubust4=abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[0]]-dual_variable_weight_average_robust4)
                for j in range(1,len(u_neighbor_help_robust4)):
                    if abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]]-dual_variable_weight_average_robust4)>=max_distance_roubust4:
                        max_distance_roubust4=abs(dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]]-dual_variable_weight_average_robust4)
                        neighbor_index_robust4=u_neighbor_help_robust4[j]
                u_neighbor_help_robust4.remove(neighbor_index_robust4)
                u_neighbor_help_robust4.append(i)
            #计算剔除q个信息后的权重平均对偶值
            dual_variable_weight_average_robust4=0
            h_robust4=0
            for j in range(0,len(u_neighbor_help_robust4)):
                h_robust4=h_robust4+W_robust4[i][u_neighbor_help_robust4[j]]
            for j in range(0,len(u_neighbor_help_robust4)):
                dual_variable_weight_average_robust4=dual_variable_weight_average_robust4+(1/h_robust4)*(W_robust4[i][u_neighbor_help_robust4[j]]*dual_variable_intermediate_robust4[u_neighbor_help_robust4[j]])     
            #诚实用户基于IOS鲁棒聚合后的值进行自身的对偶变量的更新 基于权重平均值
            dual_variable_robust4[i][k]=dual_variable_weight_average_robust4
            
            #检测是否将拜占庭用户剔除
            for j in range(0,len(u_neighbor_help_robust4)):
                if u_neighbor_help_robust4[j]==94 or u_neighbor_help_robust4[j]==95 or u_neighbor_help_robust4[j]==96 or u_neighbor_help_robust4[j]==97 or u_neighbor_help_robust4[j]==98 or u_neighbor_help_robust4[j]==99:
                    check_robust4[i][k]=1             
            
        #拜占庭用户对偶变量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            dual_variable_robust4[i][k]=random_attack3_robust4[i-94][k]
        
        #诚实用户资源申请量的迭代
        for i in range (0,100-int(proportion_robust4*100)):
            agent_resource_quantity_robust4[i][k]=(dual_variable_robust4[i][k]-2*a_robust4[i]*b_robust4[i])/(-2*a_robust4[i])
            #原变量的投影操作
            if agent_resource_quantity_robust4[i][k] < 0:
                agent_resource_quantity_robust4[i][k]=0
            elif agent_resource_quantity_robust4[i][k] > 100: 
                agent_resource_quantity_robust4[i][k]=100  
        
        #拜占庭用户资源申请量的迭代
        for i in range(100-int(proportion_robust4*100),100):
            agent_resource_quantity_robust4[i][k]=agent_resource_quantity_robust4[i][k-1]       
        
        #计算所有诚实用户的平均资源量
        sum1_robust4=0
        for i in range (0,100-int(proportion_robust4*100)):
            sum1_robust4 = sum1_robust4 + agent_resource_quantity_robust4[i][k]
        average_resource_honest_robust4[0][k]=sum1_robust4/(100-int(proportion_robust4*100))
        
        #计算所有诚实用户的平均成本
        sum2_robust4=0
        for i in range(0,100-int(proportion_robust4*100)):
            sum2_robust4=sum2_robust4+a_robust4[i]*(agent_resource_quantity_robust4[i][k]-b_robust4[i])*(agent_resource_quantity_robust4[i][k]-b_robust4[i])
        average_cost_honest_robust4[0][k]=sum2_robust4/(100-int(proportion_robust4*100))
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity_robust4[i][k])
            print("资源申请量平均值avg为:",average_resource_honest_robust4[0][k])
            print("****************************************************************************")
            print("平均成本为:",average_cost_honest_robust4[0][k])
            print("****************************************************************************")
            for i in range(0,100):
                print("agent",(i+1),"在",k,"次迭代中的对偶变量为:",dual_variable_robust4[i][k])   
            print("*************************")
            print("迭代步长为:",iteration_update_robust4)
            print("****************************************************************************")
            '''print("聚合权重为:",h_robust3)
            print("****************************************************************************")'''
        
        iteration_update_robust4=1/(pow(k+1,0.1)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [23]:
Robust_DGD_IOS_robust4(a_robust4,b_robust4,agent_resource_quantity_robust4,average_resource_honest_robust4,average_cost_honest_robust4,iteration_update_robust4,dual_variable_robust4,A_robust4,iteration_robust4,q_robust4)

****************************DGD算法+IOS+高斯拜占庭攻击****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 0.0
agent 2 的初始资源申请量为: 0.0
agent 3 的初始资源申请量为: 0.0
agent 4 的初始资源申请量为: 0.0
agent 5 的初始资源申请量为: 0.0
agent 6 的初始资源申请量为: 0.0
agent 7 的初始资源申请量为: 0.0
agent 8 的初始资源申请量为: 0.0
agent 9 的初始资源申请量为: 0.0
agent 10 的初始资源申请量为: 0.0
agent 11 的初始资源申请量为: 0.0
agent 12 的初始资源申请量为: 0.0
agent 13 的初始资源申请量为: 0.0
agent 14 的初始资源申请量为: 0.0
agent 15 的初始资源申请量为: 0.0
agent 16 的初始资源申请量为: 0.0
agent 17 的初始资源申请量为: 0.0
agent 18 的初始资源申请量为: 0.0
agent 19 的初始资源申请量为: 0.0
agent 20 的初始资源申请量为: 0.0
agent 21 的初始资源申请量为: 0.0
agent 22 的初始资源申请量为: 0.0
agent 23 的初始资源申请量为: 0.0
agent 24 的初始资源申请量为: 0.0
agent 25 的初始资源申请量为: 0.0
agent 26 的初始资源申请量为: 0.0
agent 27 的初始资源申请量为: 0.0
agent 28 的初始资源申请量为: 0.0
agent 29 的初始资源申请量为: 0.0
agent 30 的初始资源申请量为: 0.0
agent 31 的初始资源申请量为: 0.0
agent 32 的初始资源申请量为: 0.0
agent 33 的初始资源申请量为: 0.0
agent 34 的初始资源申请量为: 0.0
agent 35 的初始资源申请量为: 0.0
agent 36 的初始资源申请量为: 0.0
agent 37 的初始资源申请量为: 0.0
agent 38 的初始资源申请量为: 0.0
agent 39 的初始资

3999 次迭代中的结果：
agent 1 在 3999 次迭代中的资源申请量为: 56.12070683276905
agent 2 在 3999 次迭代中的资源申请量为: 55.95304928256159
agent 3 在 3999 次迭代中的资源申请量为: 60.561504445775654
agent 4 在 3999 次迭代中的资源申请量为: 37.87880941536425
agent 5 在 3999 次迭代中的资源申请量为: 57.485123743591
agent 6 在 3999 次迭代中的资源申请量为: 55.264203791618655
agent 7 在 3999 次迭代中的资源申请量为: 65.53125226251791
agent 8 在 3999 次迭代中的资源申请量为: 46.27086551757551
agent 9 在 3999 次迭代中的资源申请量为: 65.55042218484795
agent 10 在 3999 次迭代中的资源申请量为: 54.14963600043463
agent 11 在 3999 次迭代中的资源申请量为: 65.3573429729549
agent 12 在 3999 次迭代中的资源申请量为: 38.35910883660039
agent 13 在 3999 次迭代中的资源申请量为: 49.63453967492609
agent 14 在 3999 次迭代中的资源申请量为: 41.30791885082795
agent 15 在 3999 次迭代中的资源申请量为: 42.43389362994332
agent 16 在 3999 次迭代中的资源申请量为: 49.18225461624181
agent 17 在 3999 次迭代中的资源申请量为: 50.050423332647604
agent 18 在 3999 次迭代中的资源申请量为: 46.47809367381057
agent 19 在 3999 次迭代中的资源申请量为: 44.719362884828016
agent 20 在 3999 次迭代中的资源申请量为: 50.73037511281795
agent 21 在 3999 次迭代中的资源申请量为: 69.8324590448429
agent 22

5999 次迭代中的结果：
agent 1 在 5999 次迭代中的资源申请量为: 56.17297628108825
agent 2 在 5999 次迭代中的资源申请量为: 56.005249429242454
agent 3 在 5999 次迭代中的资源申请量为: 60.61881408119263
agent 4 在 5999 次迭代中的资源申请量为: 37.91426629347785
agent 5 在 5999 次迭代中的资源申请量为: 57.53863333527171
agent 6 在 5999 次迭代中的资源申请量为: 55.315504287381664
agent 7 在 5999 次迭代中的资源申请量为: 65.59365730930715
agent 8 在 5999 次迭代中的资源申请量为: 46.31365787605212
agent 9 在 5999 次迭代中的资源申请量为: 65.61211513602582
agent 10 在 5999 次迭代中的资源申请量为: 54.200781961994586
agent 11 在 5999 次迭代中的资源申请量为: 65.4202496312116
agent 12 在 5999 次迭代中的资源申请量为: 38.39447317853655
agent 13 在 5999 次迭代中的资源申请量为: 49.68164834947196
agent 14 在 5999 次迭代中的资源申请量为: 41.34596408257298
agent 15 在 5999 次迭代中的资源申请量为: 42.472927353235804
agent 16 在 5999 次迭代中的资源申请量为: 49.22773063371897
agent 17 在 5999 次迭代中的资源申请量为: 50.09649678214453
agent 18 在 5999 次迭代中的资源申请量为: 46.52113818982445
agent 19 在 5999 次迭代中的资源申请量为: 44.76131545941006
agent 20 在 5999 次迭代中的资源申请量为: 50.77720713463902
agent 21 在 5999 次迭代中的资源申请量为: 69.89814844393362
agent

7999 次迭代中的结果：
agent 1 在 7999 次迭代中的资源申请量为: 56.175489324357414
agent 2 在 7999 次迭代中的资源申请量为: 56.007965353906926
agent 3 在 7999 次迭代中的资源申请量为: 60.62159158212441
agent 4 在 7999 次迭代中的资源申请量为: 37.91601268471958
agent 5 在 7999 次迭代中的资源申请量为: 57.541255995392
agent 6 在 7999 次迭代中的资源申请量为: 55.31800509270703
agent 7 在 7999 次迭代中的资源申请量为: 65.5966351524103
agent 8 在 7999 次迭代中的资源申请量为: 46.31561158193708
agent 9 在 7999 次迭代中的资源申请量为: 65.61508346873212
agent 10 在 7999 次迭代中的资源申请量为: 54.20359292127057
agent 11 在 7999 次迭代中的资源申请量为: 65.4232930589376
agent 12 在 7999 次迭代中的资源申请量为: 38.39601946339723
agent 13 在 7999 次迭代中的资源申请量为: 49.68379656357857
agent 14 在 7999 次迭代中的资源申请量为: 41.347826259941925
agent 15 在 7999 次迭代中的资源申请量为: 42.47470517393577
agent 16 在 7999 次迭代中的资源申请量为: 49.229785314883415
agent 17 在 7999 次迭代中的资源申请量为: 50.09869212325308
agent 18 在 7999 次迭代中的资源申请量为: 46.523070267456085
agent 19 在 7999 次迭代中的资源申请量为: 44.76311399652334
agent 20 在 7999 次迭代中的资源申请量为: 50.77954075956235
agent 21 在 7999 次迭代中的资源申请量为: 69.90139441831356
agent 2

9999 次迭代中的结果：
agent 1 在 9999 次迭代中的资源申请量为: 56.17563138506366
agent 2 在 9999 次迭代中的资源申请量为: 56.00827401335625
agent 3 在 9999 次迭代中的资源申请量为: 60.62176524552095
agent 4 在 9999 次迭代中的资源申请量为: 37.91614275552205
agent 5 在 9999 次迭代中的资源申请量为: 57.541441861572025
agent 6 在 9999 次迭代中的资源申请量为: 55.318172297737725
agent 7 在 9999 次迭代中的资源申请量为: 65.59678655677419
agent 8 在 9999 次迭代中的资源申请量为: 46.31564401731814
agent 9 在 9999 次迭代中的资源申请量为: 65.61525293818953
agent 10 在 9999 次迭代中的资源申请量为: 54.204016573454865
agent 11 在 9999 次迭代中的资源申请量为: 65.42347936737477
agent 12 在 9999 次迭代中的资源申请量为: 38.39599103420858
agent 13 在 9999 次迭代中的资源申请量为: 49.683830204976175
agent 14 在 9999 次迭代中的资源申请量为: 41.347956362274005
agent 15 在 9999 次迭代中的资源申请量为: 42.47473129706811
agent 16 在 9999 次迭代中的资源申请量为: 49.229802355617565
agent 17 在 9999 次迭代中的资源申请量为: 50.09880132319444
agent 18 在 9999 次迭代中的资源申请量为: 46.52307609052926
agent 19 在 9999 次迭代中的资源申请量为: 44.76305130458887
agent 20 在 9999 次迭代中的资源申请量为: 50.77973436797195
agent 21 在 9999 次迭代中的资源申请量为: 69.90164390319808
ag

...

51999 次迭代中的结果：
agent 1 在 51999 次迭代中的资源申请量为: 56.17570583156672
agent 2 在 51999 次迭代中的资源申请量为: 56.00947727472043
agent 3 在 51999 次迭代中的资源申请量为: 60.621968159243025
agent 4 在 51999 次迭代中的资源申请量为: 37.916421343091805
agent 5 在 51999 次迭代中的资源申请量为: 57.54179180475173
agent 6 在 51999 次迭代中的资源申请量为: 55.318433286222785
agent 7 在 51999 次迭代中的资源申请量为: 65.59675219664979
agent 8 在 51999 次迭代中的资源申请量为: 46.31513726231826
agent 9 在 51999 次迭代中的资源申请量为: 65.61535303908309
agent 10 在 51999 次迭代中的资源申请量为: 54.20601608468502
agent 11 在 51999 次迭代中的资源申请量为: 65.42367283460275
agent 12 在 51999 次迭代中的资源申请量为: 38.39519839555732
agent 13 在 51999 次迭代中的资源申请量为: 49.683258376913756
agent 14 在 51999 次迭代中的资源申请量为: 41.34819124208708
agent 15 在 51999 次迭代中的资源申请量为: 42.474245630962805
agent 16 在 51999 次迭代中的资源申请量为: 49.229145830464255
agent 17 在 51999 次迭代中的资源申请量为: 50.09875848585519
agent 18 在 51999 次迭代中的资源申请量为: 46.522384945319395
agent 19 在 51999 次迭代中的资源申请量为: 44.76191493665629
agent 20 在 51999 次迭代中的资源申请量为: 50.78024996086717
agent 21 在 51999 次迭代中的资源申请量

53999 次迭代中的结果：
agent 1 在 53999 次迭代中的资源申请量为: 56.17570725644687
agent 2 在 53999 次迭代中的资源申请量为: 56.00950242610763
agent 3 在 53999 次迭代中的资源申请量为: 60.62197227313143
agent 4 在 53999 次迭代中的资源申请量为: 37.91642710284778
agent 5 在 53999 次迭代中的资源申请量为: 57.54179901794708
agent 6 在 53999 次迭代中的资源申请量为: 55.31843863262337
agent 7 在 53999 次迭代中的资源申请量为: 65.59675130679224
agent 8 在 53999 次迭代中的资源申请量为: 46.31512649564598
agent 9 在 53999 次迭代中的资源申请量为: 65.61535497960534
agent 10 在 53999 次迭代中的资源申请量为: 54.20605797550309
agent 11 在 53999 次迭代中的资源申请量为: 65.42367673401846
agent 12 在 53999 次迭代中的资源申请量为: 38.39518164022969
agent 13 在 53999 次迭代中的资源申请量为: 49.68324623138798
agent 14 在 53999 次迭代中的资源申请量为: 41.348196076456006
agent 15 在 53999 次迭代中的资源申请量为: 42.474235318132436
agent 16 在 53999 次迭代中的资源申请量为: 49.22913190813551
agent 17 在 53999 次迭代中的资源申请量为: 50.09875746179732
agent 18 在 53999 次迭代中的资源申请量为: 46.52237030270356
agent 19 在 53999 次迭代中的资源申请量为: 44.76189093882359
agent 20 在 53999 次迭代中的资源申请量为: 50.780260671816166
agent 21 在 53999 次迭代中的资源申请量为: 6

55999 次迭代中的结果：
agent 1 在 55999 次迭代中的资源申请量为: 56.17570862505862
agent 2 在 55999 次迭代中的资源申请量为: 56.00952657376532
agent 3 在 55999 次迭代中的资源申请量为: 60.62197622352962
agent 4 在 55999 次迭代中的资源申请量为: 37.916432632991466
agent 5 在 55999 次迭代中的资源申请量为: 57.54180594379163
agent 6 在 55999 次迭代中的资源申请量为: 55.318443766067254
agent 7 在 55999 次迭代中的资源申请量为: 65.59675045319692
agent 8 在 55999 次迭代中的资源申请量为: 46.315116159398904
agent 9 在 55999 次迭代中的资源申请量为: 65.61535684345327
agent 10 在 55999 次迭代中的资源申请量为: 54.20609819414635
agent 11 在 55999 次迭代中的资源申请量为: 65.4236804785366
agent 12 在 55999 次迭代中的资源申请量为: 38.39516555439683
agent 13 在 55999 次迭代中的资源申请量为: 49.68323457142065
agent 14 在 55999 次迭代中的资源申请量为: 41.348200718207366
agent 15 在 55999 次迭代中的资源申请量为: 42.47422541758093
agent 16 在 55999 次迭代中的资源申请量为: 49.2291185422582
agent 17 在 55999 次迭代中的资源申请量为: 50.098756479175165
agent 18 在 55999 次迭代中的资源申请量为: 46.52235624530924
agent 19 在 55999 次迭代中的资源申请量为: 44.76186789977312
agent 20 在 55999 次迭代中的资源申请量为: 50.78027095558036
agent 21 在 55999 次迭代中的资源申请量为: 6

57999 次迭代中的结果：
agent 1 在 57999 次迭代中的资源申请量为: 56.17570994149833
agent 2 在 57999 次迭代中的资源申请量为: 56.00954979149891
agent 3 在 57999 次迭代中的资源申请量为: 60.621980022414895
agent 4 在 57999 次迭代中的资源申请量为: 37.91643795038843
agent 5 在 57999 次迭代中的资源申请量为: 57.54181260338083
agent 6 在 57999 次迭代中的资源申请量为: 55.318448702183005
agent 7 在 57999 次迭代中的资源申请量为: 65.59674963314488
agent 8 在 57999 次迭代中的资源申请量为: 46.315106221874295
agent 9 在 57999 次迭代中的资源申请量为: 65.61535863621408
agent 10 在 57999 次迭代中的资源申请量为: 54.206136863601046
agent 11 在 57999 次迭代中的资源申请量为: 65.42368407949911
agent 12 在 57999 次迭代中的资源申请量为: 38.39515008877206
agent 13 在 57999 次迭代中的资源申请量为: 49.68322336124829
agent 14 在 57999 次迭代中的资源申请量为: 41.348205181482314
agent 15 在 57999 次迭代中的资源申请量为: 42.47421589894293
agent 16 在 57999 次迭代中的资源申请量为: 49.22910569185577
agent 17 在 57999 次迭代中的资源申请量为: 50.09875553490225
agent 18 在 57999 次迭代中的资源申请量为: 46.52234273004378
agent 19 在 57999 次迭代中的资源申请量为: 44.76184574892875
agent 20 在 57999 次迭代中的资源申请量为: 50.78028084355205
agent 21 在 57999 次迭代中的资源申请量为:

59999 次迭代中的结果：
agent 1 在 59999 次迭代中的资源申请量为: 56.175711209436855
agent 2 在 59999 次迭代中的资源申请量为: 56.00957214537273
agent 3 在 59999 次迭代中的资源申请量为: 60.621983680512606
agent 4 在 59999 次迭代中的资源申请量为: 37.916443070137106
agent 5 在 59999 次迭代中的资源申请量为: 57.54181901560086
agent 6 在 59999 次迭代中的资源申请量为: 55.31845345496318
agent 7 在 59999 次迭代中的资源申请量为: 65.59674884420761
agent 8 在 59999 次迭代中的资源申请量为: 46.315096654699886
agent 9 在 59999 次迭代中的资源申请量为: 65.61536036289394
agent 10 在 59999 次迭代中的资源申请量为: 54.20617409395096
agent 11 在 59999 次迭代中的资源申请量为: 65.42368754706307
agent 12 在 59999 次迭代中的资源申请量为: 38.39513519924404
agent 13 在 59999 次迭代中的资源申请量为: 49.68321256886444
agent 14 在 59999 次迭代中的资源申请量为: 41.3482094789411
agent 15 在 59999 次迭代中的资源申请量为: 42.474206735043005
agent 16 在 59999 次迭代中的资源申请量为: 49.229093320255615
agent 17 在 59999 次迭代中的资源申请量为: 50.0987546262199
agent 18 在 59999 次迭代中的资源申请量为: 46.5223297183404
agent 19 在 59999 次迭代中的资源申请量为: 44.76182442312431
agent 20 在 59999 次迭代中的资源申请量为: 50.78029036383285
agent 21 在 59999 次迭代中的资源申请量为: 6

In [24]:
np.savetxt('./data/01_IOS_oracle_agent_resource_quantity_Gaussian_attack_small.txt',agent_resource_quantity_robust4,fmt='%0.8f')#用户资源
np.savetxt('./data/01_IOS_oracle_average_cost_honest_Gaussian_attack_small.txt',average_cost_honest_robust4,fmt='%0.8f')#诚实用户平均成本
np.savetxt('./data/01_IOS_oracle_average_resource_honest_Gaussian_attack_small.txt',average_resource_honest_robust4,fmt='%0.8f')#诚实用户平均资源
np.savetxt('./data/01_IOS_oracle_dual_variable_Gaussian_attack_small.txt',dual_variable_robust4,fmt='%0.8f')#用户对偶变量